In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.signal import get_window
from smstools.models import hpsModel as HPS
from smstools.transformations import hpsTransformations as HPST
from smstools.models import utilFunctions as UF

In [ ]:
inputFile1 = "../sounds/violin-B3.wav"
fs1, x1 = UF.wavread(inputFile1)
plt.figure(0, figsize=(10, 2))
time_indexes = np.linspace(0, x1.size/fs1, x1.size)
plt.plot(time_indexes, x1)
plt.xlabel('time (seconds)')
plt.ylabel('amplitude')
plt.show()
ipd.display(ipd.Audio(data=x1, rate=fs1))

inputFile2 = "../sounds/soprano-E4.wav"
fs2, x2 = UF.wavread(inputFile2)
plt.figure(0, figsize=(10, 2))
time_indexes = np.linspace(0, x2.size/fs2, x2.size)
plt.plot(time_indexes, x2)
plt.xlabel('time (seconds)')
plt.ylabel('amplitude')
plt.show()
ipd.display(ipd.Audio(data=x2, rate=fs2))

In [ ]:
window1="blackman"
M1=1001
N1=1024
t1=-100
minSineDur1=0.05
nH=60
minf01=200
maxf01=300
f0et1=10
harmDevSlope1=0.01
stocf=0.1

window2="blackman"
M2=901
N2=1024
t2=-100
minSineDur2=0.05
minf02=250
maxf02=500
f0et2=10
harmDevSlope2=0.01

# size of fft used in synthesis
Ns = 512
# hop size (has to be 1/4 of Ns)
H = 128
# compute analysis windows
w1 = get_window(window1, M1)
w2 = get_window(window2, M2)
# compute the harmonic plus stochastic models
hfreq1, hmag1, hphase1, stocEnv1 = HPS.hpsModelAnal(x1, fs1, w1, N1, H, t1, nH, minf01, maxf01, f0et1, harmDevSlope1, minSineDur1, Ns, stocf)
hfreq2, hmag2, hphase2, stocEnv2 = HPS.hpsModelAnal(x2, fs2, w2, N2, H, t2, nH, minf02, maxf02, f0et2, harmDevSlope2, minSineDur2, Ns, stocf)

# create figure to plot
plt.figure(figsize=(10, 6))
# frequency range to plot
maxplotfreq = 15000.0

# plot spectrogram stochastic component of sound 1
plt.subplot(2, 1, 1)
numFrames = int(stocEnv1[:, 0].size)
sizeEnv = int(stocEnv1[0, :].size)
frmTime = H * np.arange(numFrames) / float(fs1)
binFreq = (0.5 * fs1) * np.arange(sizeEnv * maxplotfreq / (0.5 * fs1)) / sizeEnv
plt.pcolormesh(frmTime, binFreq, np.transpose(stocEnv1[:, : int(sizeEnv * maxplotfreq / (0.5 * fs1)) + 1]))
plt.autoscale(tight=True)

# plot harmonic on top of stochastic spectrogram of sound 1
harms = np.copy(hfreq1)
harms = harms * np.less(harms, maxplotfreq)
harms[harms == 0] = np.nan
numFrames = int(harms[:, 0].size)
frmTime = H * np.arange(numFrames) / float(fs1)
plt.plot(frmTime, harms, color="k", ms=3, alpha=1)
plt.xlabel("time (sec)")
plt.ylabel("frequency (Hz)")

# plot spectrogram stochastic component of sound 2
plt.subplot(2, 1, 2)
numFrames = int(stocEnv2[:, 0].size)
sizeEnv = int(stocEnv2[0, :].size)
frmTime = H * np.arange(numFrames) / float(fs2)
binFreq = (0.5 * fs2) * np.arange(sizeEnv * maxplotfreq / (0.5 * fs2)) / sizeEnv
plt.pcolormesh(frmTime, binFreq, np.transpose(stocEnv2[:, : int(sizeEnv * maxplotfreq / (0.5 * fs2)) + 1]))
plt.autoscale(tight=True)

# plot harmonic on top of stochastic spectrogram of sound 2
harms = np.copy(hfreq2)
harms = harms * np.less(harms, maxplotfreq)
harms[harms == 0] = np.nan
numFrames = int(harms[:, 0].size)
frmTime = H * np.arange(numFrames) / float(fs2)
plt.plot(frmTime, harms, color="k", ms=3, alpha=1)
plt.xlabel("time (sec)")
plt.ylabel("frequency (Hz)")
plt.autoscale(tight=True)
plt.show()